# ETL Project - NYC Airbnb's shown with local crime data

## Shawn Novak & Eric Mayer

### Data Analysis and Visualization Cohort 5

In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Created PostgresSQL Tables for Airbnb and NYC Crime in pgAdmin4

## Extract CSV's into DataBases

In [2]:
airbnb_file = "Resources/AB_NYC_2019.csv"
airbnb_df = pd.read_csv(airbnb_file)
airbnb_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
NYC_crime_file = "./Resources/NYPD_Complaint_Data_Historic.csv"
nyc_crime_df = pd.read_csv(NYC_crime_file)
nyc_crime_df.head()

C:\Users\jener\Anaconda3\envs\PythonWebMongo\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,101109527,12/31/2015,23:45:00,NaN,NaN,12/31/2015,113,FORGERY,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",...,44.0,INSIDE,BAR/NIGHT CLUB,NaN,NaN,1007314.0,241257.0,40.828848,-73.916661,"(40.828848333, -73.916661142)"
1,153401121,12/31/2015,23:36:00,NaN,NaN,12/31/2015,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,...,103.0,OUTSIDE,NaN,NaN,NaN,1043991.0,193406.0,40.697338,-73.784557,"(40.697338138, -73.784556739)"
2,569369778,12/31/2015,23:30:00,NaN,NaN,12/31/2015,117,DANGEROUS DRUGS,503.0,"CONTROLLED SUBSTANCE,INTENT TO",...,28.0,NaN,OTHER,NaN,NaN,999463.0,231690.0,40.802607,-73.945052,"(40.802606608, -73.945051911)"
3,968417082,12/31/2015,23:30:00,NaN,NaN,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,105.0,INSIDE,RESIDENCE-HOUSE,NaN,NaN,1060183.0,177862.0,40.654549,-73.726339,"(40.654549444, -73.726338791)"
4,641637920,12/31/2015,23:25:00,12/31/2015,23:30:00,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,13.0,FRONT OF,OTHER,NaN,NaN,987606.0,208148.0,40.738002,-73.987891,"(40.7380024, -73.98789129)"


### Transform nypd csv into new df with desired columns, rename columns and reset index.  Needed to reset borough name to lowercase using .title() so columns would match up in pgadmin4
### Hashtag reset index line while trying to groupby


In [4]:
# Create a filtered dataframe from specific columns
nypd_cols = ["CMPLNT_NUM", "OFNS_DESC", "BORO_NM"]
new_nyc_crime_df= nyc_crime_df[nypd_cols].copy()

# Rename the column headers
new_nyc_crime_df = new_nyc_crime_df.rename(columns={"CMPLNT_NUM": "complaint_id",
                                                          "OFNS_DESC": "type_offense",
                                                          "BORO_NM": "borough_name"})

# Clean the data by dropping duplicates and setting the index
new_nyc_crime_df.drop_duplicates("complaint_id", inplace=True)
new_nyc_crime_df.set_index("complaint_id", inplace=True)

new_nyc_crime_df["borough_name"] = new_nyc_crime_df["borough_name"].astype(str).str.title()

new_nyc_crime_df.head()



,type_offense,borough_name
complaint_id,,
101109527,FORGERY,Bronx
153401121,MURDER & NON-NEGL. MANSLAUGHTER,Queens
569369778,DANGEROUS DRUGS,Manhattan
968417082,ASSAULT 3 & RELATED OFFENSES,Queens
641637920,ASSAULT 3 & RELATED OFFENSES,Manhattan


### Tinkered with groupby with Quinn to try to shrink data

In [5]:
# new_nyc_crime_group = new_nyc_crime_df.groupby(["borough_name","type_offense"])["complaint_id"].count()
#  new_nyc_crime_group


### Transform airbnb csv into new df with desired columns, rename columns and reset index

In [6]:
# Create a filtered dataframe from specific columns
airbnb_cols = ["id", "name", "neighbourhood_group"]
new_airbnb_df = airbnb_df[airbnb_cols].copy()
# Rename the column headers
new_airbnb_df = new_airbnb_df.rename(columns={"id": "airbnb_id",
                                                         "name": "name",
                                                         "neighbourhood_group": "borough_name"})
# Clean the data by dropping duplicates and setting the index
new_airbnb_df.drop_duplicates("airbnb_id", inplace=True)
new_airbnb_df.set_index("airbnb_id", inplace=True)
new_airbnb_df.head()

,name,borough_name
airbnb_id,,
2539,Clean & quiet apt home by the park,Brooklyn
2595,Skylit Midtown Castle,Manhattan
3647,THE VILLAGE OF HARLEM....NEW YORK !,Manhattan
3831,Cozy Entire Floor of Brownstone,Brooklyn
5022,Entire Apt: Spacious Studio/Loft by central park,Manhattan


### Create database connection

In [7]:
connection_string = "postgres:chicago1023@localhost:5432/etl_project"
engine = create_engine(f'postgresql://{connection_string}')

In [8]:
# Confirm tables
engine.table_names()

['airbnb', 'nyc_crime']

### Load DataFrames into database

In [9]:
new_airbnb_df.to_sql(name='airbnb', con=engine, if_exists='append', index=True)

In [10]:
new_nyc_crime_df.to_sql(name='nyc_crime', con=engine, if_exists='append', index=True)

### Spoke with Quinn about needing to keep data in raw form.
### Tried to join two tables in pgAdmin4 and our files would merge on borough name and would then multiply our database to such a large, unusable file.  Learned necessity of knowing when to join tables and when such tables may not need to be joined.
### All SQL schemas located in sql.schema file(including create table, edit table & join tables)